# Project 2: Enhancer Classification Problem
Nikita Kozlov, 317099

The goal of Project 2 is to train a classifier capable of predicting enhancer sequences based on DNA sequence data using the frequency of k-mers.



In [1]:
# %conda install biopython pybedtools -y
# %pip install numpy pandas lightgbm scikit-learn tqdm matplotlib xgboost pyfaidx --upgrade

## Data Preparation

Data description:
- `experiments.tsv` contains the results of enhancer experiments with following columns:
  - curation status: Indicates whether the enhancer’s activity has been experimentally validated (positive or negative).
  - coordinate hg38: Contains the genomic coordinates for the hg38 assembly, formatted as chr16:86396481-86397120.
  - seq hg38: Provides the DNA sequence for the specified region in the hg38 genome.

- `GRCh38.p14.genome.fa` contains the human genome assembly GRCh38 in FASTA format.

Positive data: 
- Records from experiments.tsv.gz where curation `status == ’positive’`.

Negative data: 
- Records where curation `status == ’negative’`.

- Random sequences from the entire genome (GRCh38 FASTA file), ensuring:
  - No overlap with positive sequences.
  - An equal number of negative sequences to positive sequences.
  - Sequence lengths matching the lengths of positive sequences.
  - No N symbols in the sequences.

In [2]:
import numpy as np
import pandas as pd
from pyfaidx import Fasta

In [3]:
# Load the experiments data
experiments = pd.read_csv('data/experiments.tsv', sep='\t')
experiments = experiments[['vista_id', 'curation_status', 'coordinate_hg38', 'seq_hg38']].dropna()
experiments

,vista_id,curation_status,coordinate_hg38,seq_hg38
0,hs1,positive,chr16:86396481-86397120,AACTGAAGGGACCCCGTTAGCATATAAACAAAAGGTGGGGGGTAGC...
1,hs2,negative,chr16:85586489-85588130,GGCCCTGGTATGTTTGTTCTTCCAGGGGCTCCCAGGATGGATCCAG...
2,hs3,negative,chr16:80389446-80390755,AAGATTGCCATTTGGGGTGTTTCTTGGGGCTAAGAACCATGAAGAC...
3,hs4,positive,chr16:80338700-80339858,CAGAGACAGACAGTGACAGAGACAGATTTTAGAATTTGAACAAAGG...
4,hs5,negative,chr16:79936010-79937400,TGACACCCACTATTATCCAGTCCTTGATAAACCTCTTTATTTGTTC...
...,...,...,...,...
4612,mm2322,allelic,chrX:24989747-24991635,CCATGGGGGGTGGGGGTGGGTGATGAACATGTTTTCTGCTGGGGTA...
4613,mm2322,allelic,chrX:24989747-24991635,CCATGGGGGGTGGGGGTGGGTGATGAACATGTTTTCTGCTGGGGTA...
4614,mm2323,positive,chr3:157835670-157838142,ttacatgtcttttccttcttgtgtggaattctttgcggattggggt...
4615,mm2340,positive,chr14:74203486-74204485,CCTTCCCACCCTTTGCCTGGCGCTTTTTCCTCTCGAGCAGCTGGGG...


In [4]:
# Load the genome data
genome = Fasta('data/GRCh38.p14.genome.fa')

In [5]:
# Load the positive data
positive = experiments[experiments['curation_status'] == 'positive']
positive

,vista_id,curation_status,coordinate_hg38,seq_hg38
0,hs1,positive,chr16:86396481-86397120,AACTGAAGGGACCCCGTTAGCATATAAACAAAAGGTGGGGGGTAGC...
3,hs4,positive,chr16:80338700-80339858,CAGAGACAGACAGTGACAGAGACAGATTTTAGAATTTGAACAAAGG...
10,hs12,positive,chr16:78476711-78478047,AAGCTAGCTAATTGCTTCTTCAGTTGAAGACCTAAATGAGTTTTAA...
14,hs16,positive,chr16:72947001-72948646,GGGCTTCTTGCTATGTCAGCCAATCACGGGGATCCCAAGACGGTAA...
18,hs20,positive,chr16:72704669-72706250,aggcagattttgggaggaataaaaggaagcgctagagataaaaaac...
...,...,...,...,...
4609,mm2321,positive,chrX:24999050-25000866,CACCATCACCCCTTTCTCCAGCCCTTACTACCTCTTACCTCCAACA...
4611,mm2322,positive,chrX:24989747-24991635,CCATGGGGGGTGGGGGTGGGTGATGAACATGTTTTCTGCTGGGGTA...
4614,mm2323,positive,chr3:157835670-157838142,ttacatgtcttttccttcttgtgtggaattctttgcggattggggt...
4615,mm2340,positive,chr14:74203486-74204485,CCTTCCCACCCTTTGCCTGGCGCTTTTTCCTCTCGAGCAGCTGGGG...


In [6]:
# Load the negative data
negative = experiments[experiments['curation_status'] == 'negative']
negative

,vista_id,curation_status,coordinate_hg38,seq_hg38
1,hs2,negative,chr16:85586489-85588130,GGCCCTGGTATGTTTGTTCTTCCAGGGGCTCCCAGGATGGATCCAG...
2,hs3,negative,chr16:80389446-80390755,AAGATTGCCATTTGGGGTGTTTCTTGGGGCTAAGAACCATGAAGAC...
4,hs5,negative,chr16:79936010-79937400,TGACACCCACTATTATCCAGTCCTTGATAAACCTCTTTATTTGTTC...
5,hs6,negative,chr16:79916053-79917621,AGTCACCCAGGTGGTAGTGGGCTGCAGATGCTGTGGGTTTTGTTTC...
6,hs7,negative,chr16:78992666-78994265,ACAGAAGCCTCAAGCCTAACCAACAAGAAAGATCACTTCATATGCA...
...,...,...,...,...
4600,mm2313,negative,chr2:215578659-215580932,TTGTTAATTAGAAAATGCGTGAGAACCAGGAAAATTTATTATCACT...
4601,mm2314,negative,chr2:215623104-215625068,atgtaggtagggaccacgtaaggtgctctatacaatcacacagccc...
4602,mm2315,negative,chr2:215665911-215669584,TTTTTTATTGATTTTGGTGGGAAGATTTCTACCGTTGATTGTATTG...
4603,mm2316,negative,chr2:216017447-216018551,GTAGCTGAGATCACATTTTGACAATCCTATTTCAGATATCATTCAC...


## Negative data generation

In [19]:
from utils import transform_kmers_dict_to_feature_vector, count_kmers
from tqdm import tqdm

k = 4

train_sequences = pd.concat([positive.head(-400), negative.head(-400)])
test_sequences = pd.concat([positive.tail(400), negative.tail(400)])

tqdm._instances.clear()

bar = tqdm(total=len(test_sequences))
test_sequences_X = []
test_sequences_y = []

for i, row in test_sequences.iterrows():
    bar.update(1)
    sequence = row['seq_hg38']
    kmers = count_kmers(sequence, k)
    test_sequences_X.append(transform_kmers_dict_to_feature_vector(kmers, k))
    test_sequences_y.append(row['curation_status'] == 'positive' and 1 or 0)

test_sequences_X = np.array(test_sequences_X)
test_sequences_y = np.array(test_sequences_y)
bar.close()

bar = tqdm(total=len(train_sequences))
train_sequences_X = []
train_sequences_y = []

for i, row in train_sequences.iterrows():
    bar.update(1)
    sequence = row['seq_hg38']
    kmers = count_kmers(sequence, k)
    train_sequences_X.append(transform_kmers_dict_to_feature_vector(kmers, k))
    train_sequences_y.append(row['curation_status'] == 'positive' and 1 or 0)
    
train_sequences_X = np.array(train_sequences_X)
train_sequences_y = np.array(train_sequences_y)
bar.close()

100%|██████████| 3380/3380 [00:02<00:00, 1263.19it/s]


## Model Training and Validation

- Use at least one classification algorithm (e.g., Random Forest, SVM).
- Allocate the last 400 positive and the last 400 negative rows from the `experiments.tsv` file to the test set. Additionally, allocate 400 random sequences to the test set if using a random negative dataset. Do not use these test sequences during model training.
- Perform 10-fold cross-validation.
- Train the classifier for at least three different values of k (e.g., 3, 4, 5).

In [23]:
import xgboost as xgb

from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', xgb.XGBClassifier())
])

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
results = cross_val_score(pipeline, train_sequences_X, train_sequences_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

pipeline.fit(train_sequences_X, train_sequences_y)
predictions = pipeline.predict(test_sequences_X)
print(classification_report(test_sequences_y, predictions))

Accuracy: 62.78% (2.06%)
              precision    recall  f1-score   support

           0       0.55      0.37      0.44       400
           1       0.53      0.70      0.60       400

    accuracy                           0.53       800
   macro avg       0.54      0.53      0.52       800
weighted avg       0.54      0.53      0.52       800



## Positive + Random Negative Data Generation